<a href="https://colab.research.google.com/github/romanowiczadam/data-sb/blob/master/kurs_datascience/04_podstawy_sklearn/02_Metryki_klasyfikacji.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2.1.Klasyfikacja_binarna

In [0]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import sklearn

In [0]:
# dokładność klasyfikacji
# metryka accuracy

# accuracy = correct predictions / total predictions * 100

# przykład użycia accuracy
y_true = np.array([0,1,1,1,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1,1,1,])
y_pred = np.array([1,1,1,1,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1,0,1,0,0,1,1,1])

In [28]:
from sklearn.metrics import accuracy_score

accuracy_score(y_true, y_pred)

0.8666666666666667

In [65]:
results = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred})
results

,y_true,y_pred
0,0,1
1,1,1
2,1,1
3,1,1
4,1,0
5,0,0
6,0,0
7,1,1
8,0,0
9,0,0


In [66]:
# grupujemy po y_true
results = results.sort_values(by= 'y_true')
# resetujemy index
results = results.reset_index(drop=True)
results['sample'] = results.index + 1
results

,y_true,y_pred,sample
0,0,1,1
1,0,0,2
2,0,0,3
3,0,0,4
4,0,1,5
5,0,0,6
6,0,0,7
7,0,0,8
8,0,0,9
9,0,0,10


In [67]:
# tworzymy wykres
fig = make_subplots(rows = 2, cols = 1)
fig.add_trace(go.Scatter(x = results['sample'], y = results['y_true'],
                         mode = 'markers', name = 'y_true'), row = 1, col = 1)
fig.add_trace(go.Scatter(x = results['sample'], y = results['y_pred'],
                         mode = 'markers', name = 'y_pred'), row = 2, col = 1)
fig.update_layout(width = 900, height = 500, title = 'Klasyfikator binarny')
fig.show()

# 2.2.Macierz_konfuzji

In [0]:
from sklearn.metrics import  confusion_matrix
cm = confusion_matrix(y_true, y_pred)

In [69]:
import plotly.figure_factory as ff

def plot_confusion_matrix(cm):
    cm = cm[::-1]
    cm = pd.DataFrame(cm, columns= ['pred_0', 'pred_1'], index = ['true_1', 'true_0'])
    
    fig = ff.create_annotated_heatmap(z= cm.values, x=list(cm.columns), y = list(cm.index),
                                      colorscale = 'ice', showscale = True, reversescale = True)
    fig.update_layout(width = 500, height = 500, title = 'Confusion Matrix', font_size = 16)
    fig.show()

plot_confusion_matrix(cm)

In [70]:
print(cm)

[[15  3]
 [ 1 11]]


In [0]:
# rozwinięcie naszej tablicy do tablicy jednowymiarowej
cm_x = cm.ravel()

In [72]:
tn, fp, fn, tp = cm_x
# tn - true negative predykcja prawdziwie negatywna
# fp - false positive fałszywie pozytywna
# fn - false negative predykcja fałszywie negatywna
# tp - true positive predykcja prawdziwie pozytywna

print(f'TN : {tn}')
print(f'FP : {fp}')
print(f'FN : {fn}')
print(f'TP : {tp}')

TN : 15
FP : 3
FN : 1
TP : 11


In [0]:
# Metryki:

In [74]:
# błąd I rodzaju
# Type I error - False Positive Rate
fpr = fp / (fp + tn)
fpr

0.16666666666666666

In [75]:
# błąd II typu
# Type II error - False NEgative Rate
fpr = fn / (fn + tp)
fpr

0.08333333333333333

In [76]:
# Precision - ile obserwacji przewidywanych  jako pozytywne,
# jest w rzeczywistosci pozytywnych

precision = tp / (tp + fp)
print(precision)

0.7857142857142857


In [78]:
# Recall ile obserwacji z wszytkich pozytywnych sklasyfikowalibysmy jako
# pozytywne

recall = tp / (tp + fn)
print(recall)

0.9166666666666666


# 2.3.Krzywa ROC

In [79]:
from sklearn.metrics import roc_curve

# krzywa zwraca (pos_label - klasa pozytywna)
fpr, tpr, thresh = roc_curve(y_true, y_pred, pos_label =1)
roc = pd.DataFrame({'fpr': fpr, 'tpr': tpr})
roc


,fpr,tpr
0,0.000000,0.000000
1,0.166667,0.916667
2,1.000000,1.000000


In [84]:
def plot_roc_curve(y_true, y_pred):
    # Binary classyfication
    from sklearn.metrics import roc_curve
    fpr, tpr, tresh = roc_curve(y_true, y_pred, pos_label = 1)

    fig = go.Figure(data= [go.Scatter(x = roc['fpr'], y=roc['tpr'], line_color= 'red', name = 'ROC Curve'),
                    go.Scatter(x = [0, 1], y = [0, 1], mode = 'lines', line_dash = 'dash', line_color = 'navy')],
                    layout = go.Layout(xaxis_title = 'False Positive Rate',
                                       yaxis_title = 'True Positive Rate',
                                       title = 'ROC Curve',
                                       showlegend = False,
                                       width = 800))
    fig.show()
plot_roc_curve(y_true, y_pred)


# 2.4.Klasyfikacja_wieloklasowa

In [24]:
y_true_2 = np.array([0,1,0,1,1,2,0,2,0,0,0,1,1,1,1,0,2,0,1,0,0,2,2,2,2,0,1,0,2,1,2,2,1,0,0,1,2,2,2,1,0,0,0,1,0,2,1,2,0,0])
y_pred_2 = np.array([0,0,0,1,1,2,0,2,0,0,0,1,1,1,0,1,1,0,1,0,0,2,2,2,2,0,1,0,2,1,1,2,1,0,0,1,2,2,2,1,0,0,0,0,0,2,1,2,0,0])
y_true_2.shape

accuracy_score(y_true_2, y_pred_2)

0.88

In [32]:
# macierz konfuzji
cm_2 = confusion_matrix(y_true_2, y_pred_2)
cm_2

array([[19,  1,  0],
       [ 3, 12,  0],
       [ 0,  2, 13]])

In [33]:
import plotly.figure_factory as ff

def plot_confusion_matrix(cm_2):
    cm_2 = cm_2[::-1]
    cm_2 = pd.DataFrame(cm_2, columns= ['pred_0', 'pred_1', 'pred_2'], index = ['true_2','true_1', 'true_0'])
    
    fig = ff.create_annotated_heatmap(z= cm_2.values, x=list(cm_2.columns), y = list(cm_2.index),
                                      colorscale = 'ice', showscale = True, reversescale = True)
    fig.update_layout(width = 500, height = 500, title = 'Confusion Matrix', font_size = 16)
    fig.show()

plot_confusion_matrix(cm_2)

In [35]:
# raport klasyfikacji
from sklearn.metrics import classification_report

print(classification_report(y_true_2, y_pred_2))

              precision    recall  f1-score   support

           0       0.86      0.95      0.90        20
           1       0.80      0.80      0.80        15
           2       1.00      0.87      0.93        15

    accuracy                           0.88        50
   macro avg       0.89      0.87      0.88        50
weighted avg       0.89      0.88      0.88        50



In [36]:
print(classification_report(y_true_2, y_pred_2, target_names = ['label_1', 'label_2', 'label_3']))

              precision    recall  f1-score   support

     label_1       0.86      0.95      0.90        20
     label_2       0.80      0.80      0.80        15
     label_3       1.00      0.87      0.93        15

    accuracy                           0.88        50
   macro avg       0.89      0.87      0.88        50
weighted avg       0.89      0.88      0.88        50

